# Sub-Numpy
We will create our own implementation of a few functionalities supported by the NumPy
Library. We will call our implementation SNumPy (for Sub-NumPy). SNumPy will be the name of the
class you implement, and we will refer to it by the shorthand “snp” from here on

In [105]:
# Validator Class that validates the input data and ensures good error messages
class Validator:
    """
    Validator Class that validates the input data and ensures good error messages.
    """

    @staticmethod
    def is_vector(array):
        """
        Checks if the input is a vector.

        Args:
            array: The array to be checked.

        Returns:
            bool: True if the input is a vector, False otherwise.
        """
        if not isinstance(array, list):
            raise TypeError(f"Input must be a list, but received {type(array).__name__}.")

        if len(SNumPy.shape(array)) != 1:
            return False
        return True

    @staticmethod
    def is_matrix(array):
        """
        Checks if the input is a matrix (i.e., a list of lists).

        Args:
            array: The array to be checked.

        Returns:
            bool: True if the input is a matrix, False otherwise.
        """
        if not isinstance(array, list):
            raise TypeError(f"Input must be a list, but received {type(array).__name__}.")

        if len(SNumPy.shape(array)) != 2:
            return False
        return True
    
    @staticmethod
    def is_vector_or_matrix(array):
        """
        Validates whether the input is a vector or a matrix.

        Args:
            array (list): The array to be checked.

        Returns:
            bool: True if the input is a vector or matrix, False otherwise.

        Raises:
            ValueError: If the input is not a list or a mixed-type list.
        """
        if not isinstance(array, list):
            raise ValueError(f"Input must be a list, but received shape {SNumPy.shape(array)}.")

        if array and all(isinstance(item, list) for item in array):
            if not all(isinstance(item, list) for item in array):
                raise ValueError("All elements must be lists")
            if not all(isinstance(item, (int, float)) for row in array for item in row):
                raise ValueError("All elements in lists must be numeric (int or float).")
            return True  # It's a matrix
        
        elif all(not isinstance(item, list) for item in array):
            if not all(isinstance(item, (int, float)) for item in array):
                raise ValueError("All elements must be numeric (int or float).")
            return True  # It's a vector
        else:
            raise ValueError(f"Input must be either a vector or a matrix, but received shape {SNumPy.shape(array)}.")

    @staticmethod
    def validate_shape_for_operations(array1, array2, operation_type):
        """
        Validates the shape of arrays for various operations.

        Args:
            array1 (list): The first array.
            array2 (list): The second array.
            operation_type (str): The type of operation (e.g., 'add', 'multiply').

        Raises:
            ValueError: If the shapes are not compatible for the operation.
        """
        shape1 = SNumPy.shape(array1)
        shape2 = SNumPy.shape(array2)


        if operation_type in ['add', 'subtract'] and shape1 != shape2:
            raise ValueError(f"Array1 '{shape1}' and 2'{shape2}' must be of the same size for '{operation_type}' operations.")
        
        if operation_type == 'multiply' and len(shape1) == 2 and len(shape2) == 2 and shape1[1] != shape2[0]:
            raise ValueError(f"The number of columns of array1 '{shape1[1]}' must be equal to the number of rows of array2 '{shape2[0]}' for matrix multiplication.")
        
        if operation_type == 'dotproduct':
            if Validator.is_vector(array1) and Validator.is_vector(array2):
                if shape1 != shape2:
                    raise ValueError(f"Vectors must be the same size for dot product. Vector1 length: {shape1[0]}, Vector2 length: {shape2[0]}.")
            elif Validator.is_matrix(array1) and Validator.is_matrix(array2):
                if shape1[1] != shape2[0]:
                    raise ValueError(f"The number of columns in Matrix1 (shape: {shape1}) must match the number of rows in Matrix2 (shape: {shape2}) for dot product.")
            elif (Validator.is_vector(array1) and Validator.is_matrix(array2)) or (Validator.is_matrix(array1) and Validator.is_vector(array2)):
                if len(shape1) == 1:
                    if shape1[0] != shape2[0]:
                        raise ValueError(f"Vector length (length: {shape1[0]}) must match the number of rows in the matrix (shape: {shape2}) for dot product.")
                else:
                    if shape1[1] != shape2[0]:
                        raise ValueError(f"The number of columns in the matrix (shape: {shape1}) must match the vector length (length: {shape2[0]}) for dot product.")

    @staticmethod
    def validate_index_for_get(array, index):
        """
        Validates the index for accessing elements in an array.

        Args:
            array (list): The array.
            index (tuple): The index tuple (column, row).

        Raises:
            IndexError: If the index is out of bounds.
        """
        if not isinstance(index, tuple) or len(index) != 2:
            raise IndexError("Index must be a tuple of length 2.")

        column, row = index
        if row >= len(array) or (isinstance(array[0], list) and column >= len(array[0])):
            raise IndexError(f"Index {index} is out of bounds for array of shape {SNumPy.shape(array)}.")

    @staticmethod
    def validate_shape_for_reshape(array, new_shape):
        """
        Validates if a vector can be reshaped to the new shape.

        Args:
            array (list): The vector to be reshaped.
            new_shape (tuple): The new shape as a tuple (rows, columns).

        Raises:
            ValueError: If reshaping is not feasible due to incompatible sizes or if input is not a vector.
        """
        if not Validator.is_vector(array):
            raise ValueError(f"Reshape operation is only valid for vectors, but received shape {SNumPy.shape(array)}.")

        if len(new_shape) != 2:
            raise ValueError(f"New shape must be a tuple of length 2, but received shape {new_shape}.")

        total_elements = len(array)
        if total_elements != new_shape[0] * new_shape[1]:
            raise ValueError(f"Total elements mismatch for the new shape. Expected {new_shape[0] * new_shape[1]} elements, but got {total_elements}.")

    @staticmethod
    def validate_positive_integer(value):
        """
        Validates that the provided value is a positive integer.

        Args:
            value: The value to be validated.

        Raises:
            ValueError: If the value is not a positive integer.
        """
        if not isinstance(value, int) or value <= 0:
            raise ValueError(f"The value {value} is not a positive integer.")

In [106]:
# Create SNumPy class that will hold our methods for data manipulation methods,  not using numpy
# Including snp.ones(Int), snp.zeros(Int), snp.reshape(array, (row, column)), snp.shape(array), snp.append(array1, array2)
# snp.get(array, (row, column)), snp.add(array1, array1), snp.subtract(array1, array1), snp.dotproduct(array1, array1)

class SNumPy:
    """
    SNumPy class for basic array manipulations.
    """

    @staticmethod
    def ones(n, m=None):
        """
        Return an array of ones in given shape (n,m).

        Args:
            n: The number of rows. If m is None, then n is the number of columns.
            m: The number of columns (default is None).

        Returns:
            A list of ones in the given shape.
        """
        # Validate the input
        Validator.validate_positive_integer(n)
        if m is not None:
            Validator.validate_positive_integer(m)

        # Create the array with list comprehension
        if m is None:    
            return [1 for i in range(n)]
        else:
            return [[1 for i in range(m)] for j in range(n)] # If m is not none, then create a 2D array

    @staticmethod
    def zeros(n, m=None):
        """
        Return an array of zeros in given shape (n,m).

        Args:
            n: The number of rows. If m is None, then n is the number of columns.
            m: The number of columns (default is None).

        Returns:
            A list of zeros in the given shape.
        """
        # Validate the input
        Validator.validate_positive_integer(n)
        if m is not None:
            Validator.validate_positive_integer(m)
        
        # Create the array with list comprehension
        if m is None:
            return [0 for i in range(n)] 
        else:
            return [[0 for i in range(m)] for j in range(n)] # If m is not none, then create a 2D array

    @staticmethod
    def reshape(array, shape):
        """
        Return an array containing the same data with a new shape.

        Args:
            array: The array to be reshaped (vector).
            shape: The new shape as a tuple (rows, columns).

        Returns:
            Array in the new shape.
        """
        # Validate the input
        Validator.validate_shape_for_reshape(array, shape)
        
        # Unpack the shape tuple
        row, column = shape

        # Slice the vector into the new shape for each row in 'shape'
        return [array[i * column : (i + 1) * column] for i in range(row)] 

    @staticmethod
    def shape(array):
        """
        Return the shape of an array.
        """
        # Validate the input
        Validator.is_vector_or_matrix(array)

        # Can't use Validator.is_vector() or Validator.is_matrix() because it uses the shape() method and will cause a recursion error
        # Check if it's a vector (1D array)
        if not array or not isinstance(array[0], list): 
            return (len(array),)
        # Else, it's a matrix (2D array)
        else:
            return (len(array), len(array[0]))

    @staticmethod
    def append(array1, array2):
        """
        Returns a new vector/matrix that is the combination of the two input vectors/matrices.

        Args:
            array1: The first vector/matrix.
            array2: The second vector/matrix.

        Returns:
            An array that appends array2 to the right of array1.
        """
        # Vectors and Matrices can't be combined
        if len(SNumPy.shape(array1)) != len(SNumPy.shape(array2)):
            raise ValueError(
                f"Vectors and Matrices cannot be combined, got array1: {SNumPy.shape(array1)}, array2: {SNumPy.shape(array2)}")

        return array1 + array2

    @staticmethod
    def get(array, index):
        """
        Return the element at the given index.

        Args:
            array: The array to be accessed.
            index: The index tuple (column, row).

        Returns:
            The element at the given index.
        """
        # Validate the input
        Validator.validate_index_for_get(array, index)

        column, row = index  # Unpack the index tuple
        return array[row][column]

    @staticmethod
    def add(array1, array2):
        """
        Return the element-wise sum of two arrays.

        Args:
            array1: The first array (vector or matrix).
            array2: The second array.

        Returns:
            NewArray = Array1 + Array2
        """
        # Validate the input
        Validator.validate_shape_for_operations(array1, array2, 'add')


        # Check if they are vectors (1D arrays)
        if Validator.is_vector(array1) and Validator.is_vector(array2):
            # Using zip() will parrallelize the addition of the arrays
            return [element1 + element2 for element1, element2 in zip(array1, array2)]
        # Else, they are matrices (2D arrays)
        else:
            return [
                [element1 + element2 for element1, element2 in zip(row1, row2)]
                for row1, row2 in zip(array1, array2)
            ]

    @staticmethod
    def subtract(array1, array2):
        """
        Return the element-wise difference of two arrays.

        Args:
            array1: The first array (vector or matrix).
            array2: The second array (vector or matrix).

        Returns:
            NewArray = Array1 - Array2
        """
        # Validate the input
        Validator.validate_shape_for_operations(array1, array2, 'subtract')

        # Check if they are vectors (1D arrays)
        if Validator.is_vector(array1) and Validator.is_vector(array2):
            # Using zip() will parrallelize the subtraction of the arrays
            return [element1 - element2 for element1, element2 in zip(array1, array2)]
        # Else, they are matrices (2D arrays)
        else:
            return [
                [element1 - element2 for element1, element2 in zip(row1, row2)]
                for row1, row2 in zip(array1, array2)
            ]

    @staticmethod
    def dotproduct(array1, array2):
        """
        Perform the dot product operation between two vectors, two matrices, or a vector and a matrix.

        Args:
            array1: The first vector/matrix.
            array2: The second vector/matrix.

        Returns:
            The dot product as a vector or matrix.

        Raises:
            ValueError: If the dimensions are not compatible for dot product.
        """
        # Validate the input
        Validator.validate_shape_for_operations(array1, array2, 'dotproduct')

        # Get shapes for operations
        shape1 = SNumPy.shape(array1)
        shape2 = SNumPy.shape(array2)

        # Check if its vectors
        if Validator.is_vector(array1) and Validator.is_vector(array2):
            return sum(array1[i] * array2[i] for i in range(len(array1)))

        # Vector and Matrix multiplication
        elif Validator.is_vector(array1) and Validator.is_matrix(array2):
            return [sum(array1[k] * array2[k][j] for k in range(shape2[0])) for j in range(shape2[1])]

        # Matrix and Vector multiplication
        elif Validator.is_matrix(array1) and Validator.is_vector(array2):
            return [sum(array1[i][k] * array2[k] for k in range(shape1[1])) for i in range(shape1[0])]

        # Matrix multiplication
        elif Validator.is_matrix(array1) and Validator.is_matrix(array2):
            return [
                [sum(array1[i][k] * array2[k][j] for k in range(shape1[1])) for j in range(shape2[1])]
                for i in range(shape1[0])
            ]
        else:
            raise ValueError("Invalid input types for dot product")
        
        


    @staticmethod
    def scalar_multiply(array, scalar):
        """
        Return the scalar product of an array.

        Args:
            array: The array to multiply.
            scalar: The scalar to multiply by.

        Returns:
            The scalar product of the array.
        """
        # Check if its a vector
        if len(SNumPy.shape(array)) == 1:
            return [element * scalar for element in array]
        # Else, its a matrix
        else:
            return [[element * scalar for element in row] for row in array]

    @staticmethod
    def aug_matrix(matrix, vector):
        """
        Append a vector as a new column to the right of a matrix.

        Args:
            matrix: The coefficient matrix to append to.
            vector: The constant vector to append.

        Returns:
            The augmented matrix.            
        """
        if len(matrix) != len(vector):
            raise ValueError(
                "Length of vector must match the number of rows in the matrix")
        
        return [row + [vector[i]] for i, row in enumerate(matrix)]

Gaussian Elimination adopted from the code found in this video, modified to work with list and not arrays as well as using our own SNumPy methods. (StudySession, 2023)

StudySession (Director). (2023, February 11). Gauss Elimination With Partial Pivoting In Python | Numerical Methods. https://www.youtube.com/watch?v=DiZ0zSzZj1g


In [91]:
def gaussianElimination(matrix, vector, debug=False):
    """
    Solve a system of linear equations using Gaussian Elimination.

    This function applies Gaussian Elimination to solve the system of linear equations represented by the matrix equation Ax = B, where A is a square coefficient matrix, and B is a constant vector. 
    The function includes forward elimination with partial pivoting and backward substitution. 
    It checks for non-square matrices, singular matrices, and mismatched dimensions between the matrix and vector.

    Args:
        matrix : List[List[float]]
            Coefficient matrix (A), a 2D list representing the coefficients of the linear equations. Must be square (NxN).
        vector : List[float]
            Constant vector (B), a list representing the constant terms of the linear equations. Length must match the number of rows in 'matrix'.
        debug : Boolean (default is False)
            If True, prints the augmented matrix and upper triangular matrix.

    Returns:
        x : List[float]
            Solution vector (x) to the system Ax = B. Returns a list of float values representing the solution.

    Raises:
        ValueError
            If the matrix is not square, if the matrix is singular (determinant is zero), or if the dimensions of the matrix and vector do not match.
    """
    # Error checking for input dimensions
    try:
        if SNumPy.shape(vector) != SNumPy.shape(matrix[0]):
            raise ValueError(
                f"Vector must be the same size as the matrix's rows '{SNumPy.shape(matrix[0])}'")
    except ValueError as e:
        print(e)
        return None
    try:
        if SNumPy.shape(matrix[0]) != SNumPy.shape(matrix[1]):
            raise ValueError("Matrix must be a square matrix")
    except ValueError as e:
        print(e)
        return None

    # Create necessary variables
    n = len(vector)
    m = n - 1
    x = SNumPy.zeros(n)  # Initialize solution vector with zeros

    # Augmented matrix
    augmented_matrix = SNumPy.aug_matrix(matrix, vector)

    if debug:
        print(f"Augmented Matrix: \n {augmented_matrix}")

    # Forward elimination
    for i in range(n):
        # Partial pivoting for numerical stability (dealing with floating point errors)
        max_row = max(range(i, n), key=lambda r: abs(augmented_matrix[r][i]))
        # Check for singular matrix (no unique solution)
        if augmented_matrix[max_row][i] == 0.0:
            raise ValueError("Matrix is singular")

        augmented_matrix[i], augmented_matrix[max_row] = augmented_matrix[max_row], augmented_matrix[i]

        # Eliminate the entries below the current pivot
        for j in range(i + 1, n):
            scaling_factor = augmented_matrix[j][i] / augmented_matrix[i][i]
            augmented_matrix[j] = SNumPy.subtract(
                augmented_matrix[j], SNumPy.scalar_multiply(augmented_matrix[i], scaling_factor))

    if debug:
        print(f"Upper Triangular Matrix: \n {augmented_matrix}")

    # Backward substitution
    x[m] = augmented_matrix[m][n] / augmented_matrix[m][m]

    for i in range(m - 1, -1, -1):
        x[i] = augmented_matrix[i][n]
        for j in range(i + 1, n):
            x[i] = x[i] - augmented_matrix[i][j] * x[j]
        x[i] = x[i] / augmented_matrix[i][i]

    # Return the solution vector 
    solution = []
    for i in x:
        nearest_int = round(i)
        # Check if the element is within a very small range (0.0001) of the nearest integer
        if abs(i - nearest_int) < 0.0001:
            # If it's close enough, append the nearest integer to the solution list
            solution.append(nearest_int)
        else:
            # Else append the float value
            solution.append(i)

    return solution

Rounding was taken from the book "Introduction to Computation and Programming Using Python"

In [92]:
import numpy as np

matrix = [[3, 2, -4], [2, 3, 3], [5, -3, 1]]
vector = [3, 15, 14]
solution = gaussianElimination(matrix, vector)
print(solution)

[3, 1, 2]


In [112]:
# Test cases for SNumPy class

snp = SNumPy()
print(snp.ones(5))
print(snp.zeros(5))
print(snp.reshape([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], (4, 3)))
print(snp.shape([[1, 2, 3], [4, 5, 6]]))
print(snp.append([[1, 2, 3]], [[4, 5, 6]]))
print(snp.get([[1, 2, 3], [4, 5, 6]], (1, 1)))
print(snp.add([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.subtract([[1, 2, 3], [4, 5, 6]], [[1, 2, 3], [4, 5, 6]]))
print(snp.dotproduct([1, 2, 3], [[4, 5, 6], [1, 2, 3], [7, 8, 9]]))
print(snp.scalar_multiply([[1, 2, 3], [4, 5, 6]], 2))

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]
[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
(2, 3)
[[1, 2, 3], [4, 5, 6]]
5
[[2, 4, 6], [8, 10, 12]]
[[0, 0, 0], [0, 0, 0]]
[27, 33, 39]
[[2, 4, 6], [8, 10, 12]]


In [71]:
snp = SNumPy()
a = [1,2,3]
print(snp.shape(a))

(3,)
